In [150]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [121]:
def create_data_model():
    data = {}
    data['distance_matrix'] = [
    [  0, 290, 250,  230,  190,  334, 365,   40], # Dhaka
    [290,   0, 337,  453,  396,  560, 581,  244], # Syhlet
    [250, 337,   0,  495,  396,  540, 120,  240], # Chittagonj
    [230, 453, 495,    0,  360,  150, 595,  242], # Rajshahi
    [190, 396, 396,  360,    0,  356, 496,  253], # Jossore
    [334, 560, 540,  150,  356,    0, 674,  275], # Dinajpur
    [365, 581, 120,  595,  496,  674,   0,  397], # Coxsbazar
    [40,  244, 240,  242,  253,  275, 397,    0],# Narsingdi
    ]
    data['num_vehicles'] = 2
    data['demands'] = [0, 1, 2, 4, 3, 4, 8, 8]
    data['vehicle_capacities'] = [12, 18]
    data['depot'] = 0
    return data

In [122]:
#create data model
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [123]:
#create distance callback 
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]
#register callback
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [124]:
#set cost travel
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [125]:
#create demand callback 
def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]
#register demand callback
demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

In [126]:
routing.AddDimensionWithVehicleCapacity(
demand_callback_index,
0,  # null capacity slack
data['vehicle_capacities'],  # vehicle maximum capacities
True,  # start cumul to zero
'Capacity')

True

In [127]:
#search parameters
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)

In [128]:
#add print solution
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [129]:
#print solution
solution = routing.SolveWithParameters(search_parameters)
if solution : 
    print_solution(data, manager, routing, solution)

Objective: 2187
Route for vehicle 0:
 0 Load(0) ->  1 Load(1) ->  4 Load(4) ->  5 Load(8) ->  3 Load(12) ->  0 Load(12)
Distance of the route: 1422m
Load of the route: 12

Route for vehicle 1:
 0 Load(0) ->  7 Load(8) ->  2 Load(10) ->  6 Load(18) ->  0 Load(18)
Distance of the route: 765m
Load of the route: 18

Total distance of all routes: 2187m
Total load of all routes: 30


# ------------------------------------------------------------------

Soal 1 (TSP) + Penalty

In [215]:
def create_data_model():
    data = {}
    data['distance_matrix'] = [
        [0, 28, 20, 21, 14, 9],
        [28, 0, 22, 6, 22, 20],
        [20, 22, 0, 3, 17, 29],
        [21, 6, 3, 0, 14, 22],
        [14, 1, 17, 14, 0, 30],
        [9, 20, 29, 22, 30, 0],
    ]
    data['num_vehicles'] = 4
    data['vehicle_capacities'] = [3, 2, 5, 6] 
    data['demands'] = [0, 5, 5, 4, 3, 3]
    data['depot'] = 0
    return data

In [216]:
data = create_data_model()
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [217]:
#distance callback
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]
transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [218]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [219]:
#demand callback
def demands_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]
demand_callback_index = routing.RegisterUnaryTransitCallback(demands_callback)

In [220]:
#demand dimension
routing.AddDimensionWithVehicleCapacity(
demand_callback_index,
0,
data['vehicle_capacities'],
True,
'Capacity')
penalty = 100
for node in range(1, len(data['distance_matrix'])):
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

In [221]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(1)

In [222]:
#add print solution
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [223]:
solution = routing.SolveWithParameters(search_parameters)
if solution : 
    print_solution(data,manager, routing, solution)

Objective: 286
Route for vehicle 0:
 0 Load(0) ->  5 Load(3) ->  0 Load(3)
Distance of the route: 18m
Load of the route: 3

Route for vehicle 1:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 2:
 0 Load(0) ->  2 Load(5) ->  0 Load(5)
Distance of the route: 40m
Load of the route: 5

Route for vehicle 3:
 0 Load(0) ->  4 Load(3) ->  0 Load(3)
Distance of the route: 28m
Load of the route: 3

Total distance of all routes: 86m
Total load of all routes: 11


# -----------------------------------------------------------------